In [63]:
import pandas as pd
import os
import nltk
import re
# nltk.download('punkt')
nltk.download('stopwords')
# os.system('pip install contractions')

import contractions
import string
from nltk.corpus import stopwords, wordnet

from nltk.tokenize import word_tokenize, sent_tokenize

pd.set_option('display.max_colwidth', None)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matt.turner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [278]:
url = "https://raw.githubusercontent.com/mturner49/8600-deep-learning-project/main/data/CNN_Articels_clean.csv"

data = pd.read_csv(url)
data.head()

Index                                             Author  \
0      0                                 Jacopo Prisco, CNN   
1      2                              Stephanie Bailey, CNN   
2      3  Words by Stephanie Bailey, video by Zahra Jamshed   
3      4                    Paul R. La Monica, CNN Business   
4      7                                            Reuters   

        Date published  Category    Section  \
0  2021-07-15 02:46:59      news      world   
1  2021-05-12 07:52:09      news      world   
2  2021-06-16 02:51:30      news       asia   
3  2022-03-15 09:57:36  business  investing   
4  2022-03-15 11:27:02  business   business   

                                                                                            Url  \
0            https://www.cnn.com/2021/07/14/world/tusimple-autonomous-truck-spc-intl/index.html   
1           https://www.cnn.com/2021/05/12/world/ironhand-exoskeleton-glove-spc-intl/index.html   
2  https://www.cnn.com/2021/06/15/asia/swarm-robots-hong-kong-warehouse-hnk-spc-intl/index.html   
3             https://www.cnn.com/2022/03/15/investing/brics-emerging-markets-russia/index.html   
4             https://www.cnn.com/2022/03/15/business/russia-europe-sanctions-energy/index.html   

                                                                                        Headline  \
0  There's a shortage of truckers, but TuSimple thinks it has a solution: no driver needed - CNN   
1                Bioservo's robotic 'Ironhand' could protect factory workers from injuries - CNN   
2                                      This swarm of robots gets smarter the more it works - CNN   
3                  Russia is no longer an option for investors. These emerging markets are - CNN   
4                             Russian energy investment ban part of new EU sanctions wave  - CNN   

                                                                                                                                                                                                                                              Description  \
0                                                                                                                                   The e-commerce boom has exacerbated a global truck driver shortage, but could autonomous trucks help fix the problem?   
1                                            Working in a factory can mean doing the same task over and over, which could lead to chronic injury.  A battery-powered glove from Swedish company Bioservo could help workers by taking some of the strain.   
2                                                                                 In a Hong Kong warehouse, a swarm of autonomous robots works 24/7. They're not just working hard, they're working smart; as they operate, they get better at their job.   
3                                                                                                                 For many years, the world's most popular emerging markets have been the so-called BRICS: Brazil, Russia, India, China and South Africa.   
4  The European Union formally approved on Tuesday a new barrage of sanctions against Russia for its invasion of Ukraine, which include bans on investments in the Russian energy sector, luxury goods exports and imports of steel products from Russia.   

                                                                                               Keywords  \
0  world, There's a shortage of truckers, but TuSimple thinks it has a solution: no driver needed - CNN   
1                world, Bioservo's robotic 'Ironhand' could protect factory workers from injuries - CNN   
2                                       asia, This swarm of robots gets smarter the more it works - CNN   
3              investing, Russia is no longer an option for investors. These emerging markets are - CNN   
4                          business, Russian energy investment ban part of new EU sanctions

In [280]:
""" creating copy of dataframe """
df = data.copy()

In [281]:
""" want to drop all columns except for article text column """
keep_cols = ['Article text']

for cols in df.columns:
    if cols not in keep_cols:
        df = df.drop(columns=cols)

In [282]:
df.head()

Article text
0   (CNN)Right now, there's a shortage of truck drivers in the US and worldwide, exacerbated by the e-commerce boom brought on by the pandemic. One solution to the problem is autonomous trucks, and several companies are in a race to be the first to launch one. Among them is San Diego-based TuSimple.Founded in 2015, TuSimple has completed about 2 million miles of road tests with its 70 prototype trucks across the US, China and Europe. Although these are simply commercially available trucks retrofitted with its technology, TuSimple has deals in place with two of the world's largest truck manufacturers -- Navistar in the US and Traton, Volkswagen's trucking business, in Europe -- to design and build fully autonomous models, which it hopes to launch by 2024. Photos: The Yara Birkeland is what its builders call the world's first zero-emission, autonomous cargo ship. The ship is scheduled to make its first journey between two Norwegian towns before the end of the year. Click through to see more forms of transport set to transform the future.Hide Caption 1 of 13 Photos: Pictured here as a rendering, Oceanbird is a wind-powered transatlantic car carrier that cuts carbon emissions by 90%, compared to a standard car carrier.Hide Caption 2 of 13 Photos: It's not just maritime ships that are going green. Cities around the world are adopting electric ferries. Norwegian startup Zeabuz hopes its self-driving electric ferry (pictured here as a rendering) will help revive urban waterways.Hide Caption 3 of 13 Photos: In China, a new Maglev high-speed train rolls off the production line in Qingdao, east China's Shandong Province, on July 20. It has a top speed of 600 km per hour -- currently the fastest ground vehicle available globally. Hide Caption 4 of 13 Photos: Reaching speeds of up to 1,000 kilometers (620 miles) per hour, Hyperloop could be a sustainable replacement to short-haul flights. Dutch company Hardt (shown here as a rendering) started work on its Hyperloop test facility in Europe, anticipated to open in 2022.Hide Caption 5 of 13 Photos: Hyperloop is like a bullet train, without tracks and rails. Floating pods are propelled through a low-pressure steel tube using magnetic levitation. Virgin has been running tests with passengers on its XP-2 vehicle, pictured here.Hide Caption 6 of 13 Photos: Low carbon travel isn't just about switching to sustainable fuel sources -- it's also about redesigning the transport itself. A "Flying-V" plane designed by Delft's University of Technology in the Netherlands and Dutch airline KLM can cut fuel consumption by 20%. Ultimately, researchers hope to switch out the kerosene with a sustainable fuel source, like liquid hydrogen.Hide Caption 7 of 13 Photos: Hydrogen aviation could provide a sustainable solution for short and medium-haul flights. In September, Airbus unveiled plans for three hydrogen-powered, zero-emission aircraft which can carry 100 to 200 passengers. It hopes to launch the first ZEROe aircraft in 2035.Hide Caption 8 of 13 Photos: In December 2019, Canadian airline Harbour Air flew the world's first all-electric, zero-emission commercial aircraft. The six-seater seaplane was retrofitted with magniX's magni500 all-electric motor. Harbour Air -- which carries half a million passengers annually -- hopes to become the world's first all-electric airline. Hide Caption 9 of 13 Photos: In the case of electric bikes, the future is now: one retailer reported the sale of e-bikes and e-scooters had increased 230% this year. E-bikes give the user a boost to their pedaling, allowing them to go further with less effort. E-bikes are now even available on ride-share apps, like Uber.Hide Caption 10 of 13 Photos: While sleeper trains or buses might be the way most of us get some shut-eye on overnight travel, this rendering of a self-driving hotel suite from Toronto-based designer Steve Lee of Aprilli Design Studio might offer a plush alternative in the future.Hide Caption 11 of 13 Photos: I

In [283]:
""" using regex to remove any brackets and additional spaces in text """
df['Article text'] = df['Article text'].apply(lambda x: re.sub(r'\[[0-9]*\]', ' ', str(x)))
df['Article text'] = df['Article text'].apply(lambda x: re.sub(r'\s+', ' ', str(x)))

In [284]:
""" removing contractiong from text and extending them """
df['no_contractions'] = df['Article text'].apply(lambda x: [contractions.fix(word) for word in x.split()])

In [285]:
""" converting no_contractions column to single strings """
df['no_contractions_str'] = [' '.join(map(str, l)) for l in df['no_contractions']]

In [286]:
""" removing special characters and numbers from text """
df['formatted_text'] = df['Article text'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', str(x)))
df['formatted_text'] = df['formatted_text'].apply(lambda x: re.sub(r'\s+', ' ', str(x)))

In [287]:
df.head()

Article text  \
0   (CNN)Right now, there's a shortage of truck drivers in the US and worldwide, exacerbated by the e-commerce boom brought on by the pandemic. One solution to the problem is autonomous trucks, and several companies are in a race to be the first to launch one. Among them is San Diego-based TuSimple.Founded in 2015, TuSimple has completed about 2 million miles of road tests with its 70 prototype trucks across the US, China and Europe. Although these are simply commercially available trucks retrofitted with its technology, TuSimple has deals in place with two of the world's largest truck manufacturers -- Navistar in the US and Traton, Volkswagen's trucking business, in Europe -- to design and build fully autonomous models, which it hopes to launch by 2024. Photos: The Yara Birkeland is what its builders call the world's first zero-emission, autonomous cargo ship. The ship is scheduled to make its first journey between two Norwegian towns before the end of the year. Click through to see more forms of transport set to transform the future.Hide Caption 1 of 13 Photos: Pictured here as a rendering, Oceanbird is a wind-powered transatlantic car carrier that cuts carbon emissions by 90%, compared to a standard car carrier.Hide Caption 2 of 13 Photos: It's not just maritime ships that are going green. Cities around the world are adopting electric ferries. Norwegian startup Zeabuz hopes its self-driving electric ferry (pictured here as a rendering) will help revive urban waterways.Hide Caption 3 of 13 Photos: In China, a new Maglev high-speed train rolls off the production line in Qingdao, east China's Shandong Province, on July 20. It has a top speed of 600 km per hour -- currently the fastest ground vehicle available globally. Hide Caption 4 of 13 Photos: Reaching speeds of up to 1,000 kilometers (620 miles) per hour, Hyperloop could be a sustainable replacement to short-haul flights. Dutch company Hardt (shown here as a rendering) started work on its Hyperloop test facility in Europe, anticipated to open in 2022.Hide Caption 5 of 13 Photos: Hyperloop is like a bullet train, without tracks and rails. Floating pods are propelled through a low-pressure steel tube using magnetic levitation. Virgin has been running tests with passengers on its XP-2 vehicle, pictured here.Hide Caption 6 of 13 Photos: Low carbon travel isn't just about switching to sustainable fuel sources -- it's also about redesigning the transport itself. A "Flying-V" plane designed by Delft's University of Technology in the Netherlands and Dutch airline KLM can cut fuel consumption by 20%. Ultimately, researchers hope to switch out the kerosene with a sustainable fuel source, like liquid hydrogen.Hide Caption 7 of 13 Photos: Hydrogen aviation could provide a sustainable solution for short and medium-haul flights. In September, Airbus unveiled plans for three hydrogen-powered, zero-emission aircraft which can carry 100 to 200 passengers. It hopes to launch the first ZEROe aircraft in 2035.Hide Caption 8 of 13 Photos: In December 2019, Canadian airline Harbour Air flew the world's first all-electric, zero-emission commercial aircraft. The six-seater seaplane was retrofitted with magniX's magni500 all-electric motor. Harbour Air -- which carries half a million passengers annually -- hopes to become the world's first all-electric airline. Hide Caption 9 of 13 Photos: In the case of electric bikes, the future is now: one retailer reported the sale of e-bikes and e-scooters had increased 230% this year. E-bikes give the user a boost to their pedaling, allowing them to go further with less effort. E-bikes are now even available on ride-share apps, like Uber.Hide Caption 10 of 13 Photos: While sleeper trains or buses might be the way most of us get some shut-eye on overnight travel, this rendering of a self-driving hotel suite from Toronto-based designer Steve Lee of Aprilli Design Studio might offer a plush alternative in the future.Hide Caption 11 of 13 Photos

In [288]:
""" tokenize sentences using original article texts bc it contains punctuation stops """
df['tokenize_sentences'] = df['Article text'].apply(sent_tokenize)

In [289]:
""" removing stopwords from formatted text """
stop_words = set(stopwords.words('english'))
df['formatted_no_stopwords'] = df['formatted_text'].apply(lambda x: [word for word in nltk.word_tokenize(x) if word not in stop_words])

In [290]:
df.head()

Article text  \
0   (CNN)Right now, there's a shortage of truck drivers in the US and worldwide, exacerbated by the e-commerce boom brought on by the pandemic. One solution to the problem is autonomous trucks, and several companies are in a race to be the first to launch one. Among them is San Diego-based TuSimple.Founded in 2015, TuSimple has completed about 2 million miles of road tests with its 70 prototype trucks across the US, China and Europe. Although these are simply commercially available trucks retrofitted with its technology, TuSimple has deals in place with two of the world's largest truck manufacturers -- Navistar in the US and Traton, Volkswagen's trucking business, in Europe -- to design and build fully autonomous models, which it hopes to launch by 2024. Photos: The Yara Birkeland is what its builders call the world's first zero-emission, autonomous cargo ship. The ship is scheduled to make its first journey between two Norwegian towns before the end of the year. Click through to see more forms of transport set to transform the future.Hide Caption 1 of 13 Photos: Pictured here as a rendering, Oceanbird is a wind-powered transatlantic car carrier that cuts carbon emissions by 90%, compared to a standard car carrier.Hide Caption 2 of 13 Photos: It's not just maritime ships that are going green. Cities around the world are adopting electric ferries. Norwegian startup Zeabuz hopes its self-driving electric ferry (pictured here as a rendering) will help revive urban waterways.Hide Caption 3 of 13 Photos: In China, a new Maglev high-speed train rolls off the production line in Qingdao, east China's Shandong Province, on July 20. It has a top speed of 600 km per hour -- currently the fastest ground vehicle available globally. Hide Caption 4 of 13 Photos: Reaching speeds of up to 1,000 kilometers (620 miles) per hour, Hyperloop could be a sustainable replacement to short-haul flights. Dutch company Hardt (shown here as a rendering) started work on its Hyperloop test facility in Europe, anticipated to open in 2022.Hide Caption 5 of 13 Photos: Hyperloop is like a bullet train, without tracks and rails. Floating pods are propelled through a low-pressure steel tube using magnetic levitation. Virgin has been running tests with passengers on its XP-2 vehicle, pictured here.Hide Caption 6 of 13 Photos: Low carbon travel isn't just about switching to sustainable fuel sources -- it's also about redesigning the transport itself. A "Flying-V" plane designed by Delft's University of Technology in the Netherlands and Dutch airline KLM can cut fuel consumption by 20%. Ultimately, researchers hope to switch out the kerosene with a sustainable fuel source, like liquid hydrogen.Hide Caption 7 of 13 Photos: Hydrogen aviation could provide a sustainable solution for short and medium-haul flights. In September, Airbus unveiled plans for three hydrogen-powered, zero-emission aircraft which can carry 100 to 200 passengers. It hopes to launch the first ZEROe aircraft in 2035.Hide Caption 8 of 13 Photos: In December 2019, Canadian airline Harbour Air flew the world's first all-electric, zero-emission commercial aircraft. The six-seater seaplane was retrofitted with magniX's magni500 all-electric motor. Harbour Air -- which carries half a million passengers annually -- hopes to become the world's first all-electric airline. Hide Caption 9 of 13 Photos: In the case of electric bikes, the future is now: one retailer reported the sale of e-bikes and e-scooters had increased 230% this year. E-bikes give the user a boost to their pedaling, allowing them to go further with less effort. E-bikes are now even available on ride-share apps, like Uber.Hide Caption 10 of 13 Photos: While sleeper trains or buses might be the way most of us get some shut-eye on overnight travel, this rendering of a self-driving hotel suite from Toronto-based designer Steve Lee of Aprilli Design Studio might offer a plush alternative in the future.Hide Caption 11 of 13 Photos

In [291]:
counter = Counter()
df['formatted_no_stopwords'].map(counter.update)
most_frequent = pd.DataFrame(counter.most_common())
most_frequent.columns = ['token', 'count']
most_frequent.head(20)

,token,count
0,I,23985
1,The,23833
2,said,19513
3,CNN,9743
4,people,8086
5,It,7875
6,one,7548
7,year,7164
8,first,7096
9,time,6771


In [304]:
""" function to create dict of words and their frequencies """
def word_freq(x):
    word_frequencies = {}

    for word in x['formatted_no_stopwords']:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1
    return word_frequencies

In [313]:
df['word_freq'] = df.apply(word_freq, axis=1)

In [306]:
df.head()

Article text  \
0   (CNN)Right now, there's a shortage of truck drivers in the US and worldwide, exacerbated by the e-commerce boom brought on by the pandemic. One solution to the problem is autonomous trucks, and several companies are in a race to be the first to launch one. Among them is San Diego-based TuSimple.Founded in 2015, TuSimple has completed about 2 million miles of road tests with its 70 prototype trucks across the US, China and Europe. Although these are simply commercially available trucks retrofitted with its technology, TuSimple has deals in place with two of the world's largest truck manufacturers -- Navistar in the US and Traton, Volkswagen's trucking business, in Europe -- to design and build fully autonomous models, which it hopes to launch by 2024. Photos: The Yara Birkeland is what its builders call the world's first zero-emission, autonomous cargo ship. The ship is scheduled to make its first journey between two Norwegian towns before the end of the year. Click through to see more forms of transport set to transform the future.Hide Caption 1 of 13 Photos: Pictured here as a rendering, Oceanbird is a wind-powered transatlantic car carrier that cuts carbon emissions by 90%, compared to a standard car carrier.Hide Caption 2 of 13 Photos: It's not just maritime ships that are going green. Cities around the world are adopting electric ferries. Norwegian startup Zeabuz hopes its self-driving electric ferry (pictured here as a rendering) will help revive urban waterways.Hide Caption 3 of 13 Photos: In China, a new Maglev high-speed train rolls off the production line in Qingdao, east China's Shandong Province, on July 20. It has a top speed of 600 km per hour -- currently the fastest ground vehicle available globally. Hide Caption 4 of 13 Photos: Reaching speeds of up to 1,000 kilometers (620 miles) per hour, Hyperloop could be a sustainable replacement to short-haul flights. Dutch company Hardt (shown here as a rendering) started work on its Hyperloop test facility in Europe, anticipated to open in 2022.Hide Caption 5 of 13 Photos: Hyperloop is like a bullet train, without tracks and rails. Floating pods are propelled through a low-pressure steel tube using magnetic levitation. Virgin has been running tests with passengers on its XP-2 vehicle, pictured here.Hide Caption 6 of 13 Photos: Low carbon travel isn't just about switching to sustainable fuel sources -- it's also about redesigning the transport itself. A "Flying-V" plane designed by Delft's University of Technology in the Netherlands and Dutch airline KLM can cut fuel consumption by 20%. Ultimately, researchers hope to switch out the kerosene with a sustainable fuel source, like liquid hydrogen.Hide Caption 7 of 13 Photos: Hydrogen aviation could provide a sustainable solution for short and medium-haul flights. In September, Airbus unveiled plans for three hydrogen-powered, zero-emission aircraft which can carry 100 to 200 passengers. It hopes to launch the first ZEROe aircraft in 2035.Hide Caption 8 of 13 Photos: In December 2019, Canadian airline Harbour Air flew the world's first all-electric, zero-emission commercial aircraft. The six-seater seaplane was retrofitted with magniX's magni500 all-electric motor. Harbour Air -- which carries half a million passengers annually -- hopes to become the world's first all-electric airline. Hide Caption 9 of 13 Photos: In the case of electric bikes, the future is now: one retailer reported the sale of e-bikes and e-scooters had increased 230% this year. E-bikes give the user a boost to their pedaling, allowing them to go further with less effort. E-bikes are now even available on ride-share apps, like Uber.Hide Caption 10 of 13 Photos: While sleeper trains or buses might be the way most of us get some shut-eye on overnight travel, this rendering of a self-driving hotel suite from Toronto-based designer Steve Lee of Aprilli Design Studio might offer a plush alternative in the future.Hide Caption 11 of 13 Photos

In [312]:
""" created copy of word_freq column to use to calculate weighted frequency """
df['weighted_freq'] = df['word_freq']

In [301]:
""" getting weighted freq by diving the word frequency by the max word frequency """
def weighted_freq(x):
    maximum_frequency = max(x['weighted_freq'].values())
    for word in x['weighted_freq'].keys():
        x['weighted_freq'][word] = (x['weighted_freq'][word]/maximum_frequency)

In [310]:
df['weighted_freq'] = df.apply(weighted_freq, axis=1)

In [314]:
df.head()

Article text  \
0   (CNN)Right now, there's a shortage of truck drivers in the US and worldwide, exacerbated by the e-commerce boom brought on by the pandemic. One solution to the problem is autonomous trucks, and several companies are in a race to be the first to launch one. Among them is San Diego-based TuSimple.Founded in 2015, TuSimple has completed about 2 million miles of road tests with its 70 prototype trucks across the US, China and Europe. Although these are simply commercially available trucks retrofitted with its technology, TuSimple has deals in place with two of the world's largest truck manufacturers -- Navistar in the US and Traton, Volkswagen's trucking business, in Europe -- to design and build fully autonomous models, which it hopes to launch by 2024. Photos: The Yara Birkeland is what its builders call the world's first zero-emission, autonomous cargo ship. The ship is scheduled to make its first journey between two Norwegian towns before the end of the year. Click through to see more forms of transport set to transform the future.Hide Caption 1 of 13 Photos: Pictured here as a rendering, Oceanbird is a wind-powered transatlantic car carrier that cuts carbon emissions by 90%, compared to a standard car carrier.Hide Caption 2 of 13 Photos: It's not just maritime ships that are going green. Cities around the world are adopting electric ferries. Norwegian startup Zeabuz hopes its self-driving electric ferry (pictured here as a rendering) will help revive urban waterways.Hide Caption 3 of 13 Photos: In China, a new Maglev high-speed train rolls off the production line in Qingdao, east China's Shandong Province, on July 20. It has a top speed of 600 km per hour -- currently the fastest ground vehicle available globally. Hide Caption 4 of 13 Photos: Reaching speeds of up to 1,000 kilometers (620 miles) per hour, Hyperloop could be a sustainable replacement to short-haul flights. Dutch company Hardt (shown here as a rendering) started work on its Hyperloop test facility in Europe, anticipated to open in 2022.Hide Caption 5 of 13 Photos: Hyperloop is like a bullet train, without tracks and rails. Floating pods are propelled through a low-pressure steel tube using magnetic levitation. Virgin has been running tests with passengers on its XP-2 vehicle, pictured here.Hide Caption 6 of 13 Photos: Low carbon travel isn't just about switching to sustainable fuel sources -- it's also about redesigning the transport itself. A "Flying-V" plane designed by Delft's University of Technology in the Netherlands and Dutch airline KLM can cut fuel consumption by 20%. Ultimately, researchers hope to switch out the kerosene with a sustainable fuel source, like liquid hydrogen.Hide Caption 7 of 13 Photos: Hydrogen aviation could provide a sustainable solution for short and medium-haul flights. In September, Airbus unveiled plans for three hydrogen-powered, zero-emission aircraft which can carry 100 to 200 passengers. It hopes to launch the first ZEROe aircraft in 2035.Hide Caption 8 of 13 Photos: In December 2019, Canadian airline Harbour Air flew the world's first all-electric, zero-emission commercial aircraft. The six-seater seaplane was retrofitted with magniX's magni500 all-electric motor. Harbour Air -- which carries half a million passengers annually -- hopes to become the world's first all-electric airline. Hide Caption 9 of 13 Photos: In the case of electric bikes, the future is now: one retailer reported the sale of e-bikes and e-scooters had increased 230% this year. E-bikes give the user a boost to their pedaling, allowing them to go further with less effort. E-bikes are now even available on ride-share apps, like Uber.Hide Caption 10 of 13 Photos: While sleeper trains or buses might be the way most of us get some shut-eye on overnight travel, this rendering of a self-driving hotel suite from Toronto-based designer Steve Lee of Aprilli Design Studio might offer a plush alternative in the future.Hide Caption 11 of 13 Photos

In [315]:
""" assigning a score to each sentence based on the weighted frequency of the words """
def sentence_scores(x):
    sentence_scores = {}
    
    for sent in x['tokenize_sentences']:
        for word in nltk.word_tokenize(sent.lower()):
            if word in x['weighted_freq'].keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = x['weighted_freq'][word]
                    else:
                        sentence_scores[sent] += x['weighted_freq'][word]
    
    return sentence_scores

In [316]:
df['sentence_scores'] = df.apply(sentence_scores, axis=1)

In [317]:
df.head()

Article text  \
0   (CNN)Right now, there's a shortage of truck drivers in the US and worldwide, exacerbated by the e-commerce boom brought on by the pandemic. One solution to the problem is autonomous trucks, and several companies are in a race to be the first to launch one. Among them is San Diego-based TuSimple.Founded in 2015, TuSimple has completed about 2 million miles of road tests with its 70 prototype trucks across the US, China and Europe. Although these are simply commercially available trucks retrofitted with its technology, TuSimple has deals in place with two of the world's largest truck manufacturers -- Navistar in the US and Traton, Volkswagen's trucking business, in Europe -- to design and build fully autonomous models, which it hopes to launch by 2024. Photos: The Yara Birkeland is what its builders call the world's first zero-emission, autonomous cargo ship. The ship is scheduled to make its first journey between two Norwegian towns before the end of the year. Click through to see more forms of transport set to transform the future.Hide Caption 1 of 13 Photos: Pictured here as a rendering, Oceanbird is a wind-powered transatlantic car carrier that cuts carbon emissions by 90%, compared to a standard car carrier.Hide Caption 2 of 13 Photos: It's not just maritime ships that are going green. Cities around the world are adopting electric ferries. Norwegian startup Zeabuz hopes its self-driving electric ferry (pictured here as a rendering) will help revive urban waterways.Hide Caption 3 of 13 Photos: In China, a new Maglev high-speed train rolls off the production line in Qingdao, east China's Shandong Province, on July 20. It has a top speed of 600 km per hour -- currently the fastest ground vehicle available globally. Hide Caption 4 of 13 Photos: Reaching speeds of up to 1,000 kilometers (620 miles) per hour, Hyperloop could be a sustainable replacement to short-haul flights. Dutch company Hardt (shown here as a rendering) started work on its Hyperloop test facility in Europe, anticipated to open in 2022.Hide Caption 5 of 13 Photos: Hyperloop is like a bullet train, without tracks and rails. Floating pods are propelled through a low-pressure steel tube using magnetic levitation. Virgin has been running tests with passengers on its XP-2 vehicle, pictured here.Hide Caption 6 of 13 Photos: Low carbon travel isn't just about switching to sustainable fuel sources -- it's also about redesigning the transport itself. A "Flying-V" plane designed by Delft's University of Technology in the Netherlands and Dutch airline KLM can cut fuel consumption by 20%. Ultimately, researchers hope to switch out the kerosene with a sustainable fuel source, like liquid hydrogen.Hide Caption 7 of 13 Photos: Hydrogen aviation could provide a sustainable solution for short and medium-haul flights. In September, Airbus unveiled plans for three hydrogen-powered, zero-emission aircraft which can carry 100 to 200 passengers. It hopes to launch the first ZEROe aircraft in 2035.Hide Caption 8 of 13 Photos: In December 2019, Canadian airline Harbour Air flew the world's first all-electric, zero-emission commercial aircraft. The six-seater seaplane was retrofitted with magniX's magni500 all-electric motor. Harbour Air -- which carries half a million passengers annually -- hopes to become the world's first all-electric airline. Hide Caption 9 of 13 Photos: In the case of electric bikes, the future is now: one retailer reported the sale of e-bikes and e-scooters had increased 230% this year. E-bikes give the user a boost to their pedaling, allowing them to go further with less effort. E-bikes are now even available on ride-share apps, like Uber.Hide Caption 10 of 13 Photos: While sleeper trains or buses might be the way most of us get some shut-eye on overnight travel, this rendering of a self-driving hotel suite from Toronto-based designer Steve Lee of Aprilli Design Studio might offer a plush alternative in the future.Hide Caption 11 of 13 Photos

In [318]:
""" generate summary by using heapq.nlargest to select the 5 sentences with the highest sentence scores """
import heapq

def summary(x):
    
    summary_sentences = heapq.nlargest(5, x['sentence_scores'].keys(), key=x['sentence_scores'].get)
    
    summary = ' '.join(summary_sentences)
    return summary

In [319]:
df['summary'] = df.apply(summary, axis=1)

In [320]:
df.head()

Article text  \
0   (CNN)Right now, there's a shortage of truck drivers in the US and worldwide, exacerbated by the e-commerce boom brought on by the pandemic. One solution to the problem is autonomous trucks, and several companies are in a race to be the first to launch one. Among them is San Diego-based TuSimple.Founded in 2015, TuSimple has completed about 2 million miles of road tests with its 70 prototype trucks across the US, China and Europe. Although these are simply commercially available trucks retrofitted with its technology, TuSimple has deals in place with two of the world's largest truck manufacturers -- Navistar in the US and Traton, Volkswagen's trucking business, in Europe -- to design and build fully autonomous models, which it hopes to launch by 2024. Photos: The Yara Birkeland is what its builders call the world's first zero-emission, autonomous cargo ship. The ship is scheduled to make its first journey between two Norwegian towns before the end of the year. Click through to see more forms of transport set to transform the future.Hide Caption 1 of 13 Photos: Pictured here as a rendering, Oceanbird is a wind-powered transatlantic car carrier that cuts carbon emissions by 90%, compared to a standard car carrier.Hide Caption 2 of 13 Photos: It's not just maritime ships that are going green. Cities around the world are adopting electric ferries. Norwegian startup Zeabuz hopes its self-driving electric ferry (pictured here as a rendering) will help revive urban waterways.Hide Caption 3 of 13 Photos: In China, a new Maglev high-speed train rolls off the production line in Qingdao, east China's Shandong Province, on July 20. It has a top speed of 600 km per hour -- currently the fastest ground vehicle available globally. Hide Caption 4 of 13 Photos: Reaching speeds of up to 1,000 kilometers (620 miles) per hour, Hyperloop could be a sustainable replacement to short-haul flights. Dutch company Hardt (shown here as a rendering) started work on its Hyperloop test facility in Europe, anticipated to open in 2022.Hide Caption 5 of 13 Photos: Hyperloop is like a bullet train, without tracks and rails. Floating pods are propelled through a low-pressure steel tube using magnetic levitation. Virgin has been running tests with passengers on its XP-2 vehicle, pictured here.Hide Caption 6 of 13 Photos: Low carbon travel isn't just about switching to sustainable fuel sources -- it's also about redesigning the transport itself. A "Flying-V" plane designed by Delft's University of Technology in the Netherlands and Dutch airline KLM can cut fuel consumption by 20%. Ultimately, researchers hope to switch out the kerosene with a sustainable fuel source, like liquid hydrogen.Hide Caption 7 of 13 Photos: Hydrogen aviation could provide a sustainable solution for short and medium-haul flights. In September, Airbus unveiled plans for three hydrogen-powered, zero-emission aircraft which can carry 100 to 200 passengers. It hopes to launch the first ZEROe aircraft in 2035.Hide Caption 8 of 13 Photos: In December 2019, Canadian airline Harbour Air flew the world's first all-electric, zero-emission commercial aircraft. The six-seater seaplane was retrofitted with magniX's magni500 all-electric motor. Harbour Air -- which carries half a million passengers annually -- hopes to become the world's first all-electric airline. Hide Caption 9 of 13 Photos: In the case of electric bikes, the future is now: one retailer reported the sale of e-bikes and e-scooters had increased 230% this year. E-bikes give the user a boost to their pedaling, allowing them to go further with less effort. E-bikes are now even available on ride-share apps, like Uber.Hide Caption 10 of 13 Photos: While sleeper trains or buses might be the way most of us get some shut-eye on overnight travel, this rendering of a self-driving hotel suite from Toronto-based designer Steve Lee of Aprilli Design Studio might offer a plush alternative in the future.Hide Caption 11 of 13 Photos

In [321]:
df.to_csv('text-summarization.csv')